# 输出格式控制实验

基于吴恩达 x OpenAI 提示词工程课程的实践 notebook

## 📚 学习目标

学习如何在提示词中明确指定输出格式（XML、JSON、HTML），确保模型输出符合程序化处理需求。

## 💡 为什么需要格式控制？

大模型默认倾向于使用 **Markdown 格式**输出（因为训练数据中 Markdown 内容占比高），但在实际工程中，我们常需要：

- **结构化数据**：便于程序解析和处理
- **格式一致性**：确保每次输出都符合预期
- **API 对接**：直接传递给下游系统

## 🎯 三种常用格式对比

| 格式 | 优势 | 适用场景 | 推荐度 |
|-----|------|---------|-------|
| **XML** | • 层次结构清晰<br>• 标签语义化强<br>• Claude 官方推荐 | • 文档解析<br>• 多层嵌套数据<br>• 复杂结构 | ⭐⭐⭐⭐⭐ |
| **JSON** | • 轻量易解析<br>• API 友好<br>• 广泛支持 | • 数据交换<br>• API 集成<br>• 程序处理 | ⭐⭐⭐⭐ |
| **HTML** | • 直接可视化<br>• 富文本支持 | • 内容展示<br>• 邮件模板<br>• 网页生成 | ⭐⭐⭐ |

## 🔑 核心策略

1. **单样本提示（One-shot）**：提供一个标准示例
2. **少样本提示（Few-shot）**：提供2-3个示例覆盖不同情况
3. **明确禁止额外输出**：要求只返回格式化数据，不要解释

## 🤖 测试模型

使用 **Qwen3 8B** 模型（Q4_K_M 量化）

---

## 第一步：环境准备

安装并启动 Ollama 服务（如果已经运行过其他 notebook，可以跳过）

In [1]:
# === 1. 在 Colab 安装并启动 Ollama ===
import subprocess
import time
import os

print("⬇️ 正在安装 Ollama...")
install_cmd = "curl -fsSL https://ollama.com/install.sh | sh"
install_result = subprocess.run(install_cmd, shell=True, check=False)
if install_result.returncode == 0:
    print("✅ Ollama 安装完成")
else:
    print("⚠️ Ollama 可能已安装，继续...")

print("🚀 正在后台启动 Ollama 服务...")
ollama_server = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print("✅ Ollama 服务已在 localhost:11434 运行")

⬇️ 正在安装 Ollama...
✅ Ollama 安装完成
🚀 正在后台启动 Ollama 服务...
✅ Ollama 服务已在 localhost:11434 运行


In [2]:
# === 2. 下载模型并导入核心函数 ===
import subprocess
import requests
import json

print("⬇️ 正在下载 qwen3:8b...")
pull_result = subprocess.run(["ollama", "pull", "qwen3:8b"], check=False)
if pull_result.returncode != 0:
    raise RuntimeError("❌ qwen3:8b 下载失败")
print("✅ 模型下载完成！")

# 定义与模型交互的函数
OLLAMA_URL = "http://localhost:11434"

def get_completion(prompt, model="qwen3:8b", temperature=0.7):
    """调用模型获取回复"""
    url = f"{OLLAMA_URL}/api/chat"
    options = {
        "temperature": temperature,
        "top_p": 0.9,
        "num_ctx": 8192,
        "num_predict": 4096
    }
    messages = [{"role": "user", "content": prompt}]
    payload = {"model": model, "messages": messages, "stream": True, "options": options}

    try:
        response = requests.post(url, json=payload, stream=True)
        response.raise_for_status()
        result = ""
        print("🤖 模型回复: ", end='', flush=True)
        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line.decode('utf-8'))
                    if 'message' in data and 'content' in data['message']:
                        content = data['message']['content']
                        if content:
                            print(content, end='', flush=True)
                            result += content
                    if data.get('done', False):
                        print()
                        break
                except json.JSONDecodeError:
                    continue
        return result.strip()
    except Exception as e:
        print(f"\n❌ 错误: {str(e)}")
        return f"[错误: {str(e)}]"

print("✅ 核心函数加载完成")

⬇️ 正在下载 qwen3:8b...
✅ 模型下载完成！
✅ 核心函数加载完成


---

# 实验一：XML 格式输出（推荐）

## 📖 任务说明

**任务**：分析文档内容，以 XML 格式返回主题和要点

**为什么推荐 XML？**
- ✅ 层次结构清晰，标签语义化强
- ✅ 减少歧义（标签名即含义）
- ✅ Claude 官方推荐格式
- ✅ 适合复杂嵌套结构

## 🎯 实验设计

使用 **One-shot** 策略：提供一个标准 XML 示例，让模型模仿格式

In [3]:
# 实验一：XML 格式输出

# 测试文档
test_document = """
人工智能的发展正在深刻改变各行各业。在医疗领域，AI 辅助诊断系统能够快速分析医学影像，
提高诊断准确率。在教育领域，个性化学习系统根据学生的学习进度和特点，提供定制化的教学内容。
在交通领域，自动驾驶技术正在逐步成熟，有望减少交通事故。然而，AI 的发展也带来了隐私保护、
就业冲击等挑战，需要社会各界共同应对。
"""

# One-shot 提示：提供一个标准 XML 格式示例
xml_prompt = f"""请分析以下文档内容，提取主题和关键要点。

[示例]
文档：区块链技术具有去中心化、不可篡改的特点。它在金融领域可用于跨境支付，在供应链管理中可以追溯产品来源。
输出：
<analysis>
  <main_topic>区块链技术及应用</main_topic>
  <key_points>
    <point>去中心化、不可篡改特性</point>
    <point>金融领域：跨境支付</point>
    <point>供应链：产品溯源</point>
  </key_points>
</analysis>

现在请分析：
<document>{test_document}</document>

重要：只输出 XML 格式结果，不要包含任何解释文字。

输出："""

print("="*80)
print("实验一：XML 格式输出")
print("="*80)
print(f"测试文档：\n{test_document.strip()}\n")
print("="*80)
result_xml = get_completion(xml_prompt)
print("\n" + "="*80)
print("✅ XML 格式输出完成")

实验一：XML 格式输出
测试文档：
人工智能的发展正在深刻改变各行各业。在医疗领域，AI 辅助诊断系统能够快速分析医学影像，
提高诊断准确率。在教育领域，个性化学习系统根据学生的学习进度和特点，提供定制化的教学内容。
在交通领域，自动驾驶技术正在逐步成熟，有望减少交通事故。然而，AI 的发展也带来了隐私保护、
就业冲击等挑战，需要社会各界共同应对。

🤖 模型回复: <analysis>
  <main_topic>人工智能的发展及影响</main_topic>
  <key_points>
    <point>医疗领域：AI辅助诊断系统提升准确率</point>
    <point>教育领域：个性化学习系统定制教学内容</point>
    <point>交通领域：自动驾驶技术减少交通事故</point>
    <point>挑战：隐私保护与就业冲击</point>
  </key_points>
</analysis>

✅ XML 格式输出完成


---

# 实验二：JSON 格式输出

## 📖 任务说明

**任务**：从用户信息中提取结构化数据，以 JSON 格式返回

**为什么使用 JSON？**
- ✅ 轻量级，易于解析
- ✅ API 对接友好（RESTful API 标准格式）
- ✅ 所有编程语言都有原生支持
- ✅ 适合数据交换和存储

## 🎯 实验设计

使用 **One-shot** 策略，并明确禁止 Markdown 代码块标记

⚠️ **关键要点**：很多模型会自动在 JSON 外加 \`\`\`json ... \`\`\`，需要明确禁止

In [4]:
# 实验二：JSON 格式输出

# 测试用户信息
test_user_info = """
张伟，今年32岁，是一名高级软件工程师。他精通 Python、JavaScript 和 Go 语言，
有8年的后端开发经验。目前在北京一家互联网公司工作，负责微服务架构设计。
业余爱好是摄影和徒步旅行。
"""

# One-shot 提示：提供 JSON 示例，并明确禁止 Markdown 标记
json_prompt = f"""提取用户信息，以 JSON 格式返回。

[示例]
输入：李娜，28岁，产品经理，擅长用户体验设计和数据分析，5年工作经验，上海。
输出：
{{
  "name": "李娜",
  "age": 28,
  "occupation": "产品经理",
  "skills": ["用户体验设计", "数据分析"],
  "experience_years": 5,
  "location": "上海"
}}

重要规则：
1. 只返回纯 JSON 对象
2. 不要使用 Markdown 代码块标记（不要 ```json 和 ```）
3. 不要添加任何解释文字

现在处理：
{test_user_info}

输出："""

print("="*80)
print("实验二：JSON 格式输出")
print("="*80)
print(f"测试用户信息：\n{test_user_info.strip()}\n")
print("="*80)
result_json = get_completion(json_prompt)
print("\n" + "="*80)

# 尝试解析 JSON 验证格式正确性
try:
    import json
    # 移除可能的 Markdown 标记
    clean_json = result_json.replace('```json', '').replace('```', '').strip()
    parsed = json.loads(clean_json)
    print("✅ JSON 格式验证通过")
    print(f"📊 解析结果：{json.dumps(parsed, ensure_ascii=False, indent=2)}")
except json.JSONDecodeError as e:
    print(f"⚠️ JSON 解析失败: {e}")
    print("提示：可能需要在提示词中进一步强调格式要求")

实验二：JSON 格式输出
测试用户信息：
张伟，今年32岁，是一名高级软件工程师。他精通 Python、JavaScript 和 Go 语言，
有8年的后端开发经验。目前在北京一家互联网公司工作，负责微服务架构设计。
业余爱好是摄影和徒步旅行。

🤖 模型回复: {
  "name": "张伟",
  "age": 32,
  "occupation": "高级软件工程师",
  "skills": ["Python", "JavaScript", "Go 语言"],
  "experience_years": 8,
  "location": "北京"
}

✅ JSON 格式验证通过
📊 解析结果：{
  "name": "张伟",
  "age": 32,
  "occupation": "高级软件工程师",
  "skills": [
    "Python",
    "JavaScript",
    "Go 语言"
  ],
  "experience_years": 8,
  "location": "北京"
}


---

# 实验三：Few-shot 提升 - 复杂情感分析

## 📖 任务说明

**任务**：从用户评论中提取情感和关键词，以 XML 格式返回

**复杂度提升**：
- 需要判断细粒度情感（不仅是积极/消极/中立）
- 需要提取多个关键词
- 需要处理混合情感的评论

## 🎯 实验设计

使用 **Few-shot** 策略（2个示例）：
- **示例1**：中立情感，多个关键词
- **示例2**：消极情感，单个关键词

通过多样化示例，让模型学会处理不同情况

In [5]:
# 实验三：Few-shot 复杂情感分析

# 测试评论（包含混合情感）
test_review = "价格太贵了，完全不值这个价！不过客服态度还不错，问题解决得挺快。"

# Few-shot 提示：提供2个不同类型的示例
few_shot_prompt = f"""提取评论中的情感和关键词，以 XML 格式返回。

[示例 1]
输入：这家餐厅的服务态度很好，但菜品一般。
输出：
<review>
  <sentiment>中立</sentiment>
  <keywords>
    <keyword>服务态度</keyword>
    <keyword>菜品</keyword>
  </keywords>
</review>

[示例 2]
输入：电池续航太差了，用不到半天就要充电！
输出：
<review>
  <sentiment>消极</sentiment>
  <keywords>
    <keyword>电池续航</keyword>
  </keywords>
</review>

重要：只返回 XML 格式，不要添加解释。

现在请处理：{test_review}

输出："""

print("="*80)
print("实验三：Few-shot 复杂情感分析")
print("="*80)
print(f"测试评论：\n{test_review}\n")
print("="*80)
result_few_shot = get_completion(few_shot_prompt)
print("\n" + "="*80)
print("✅ Few-shot 情感分析完成")
print("\n💡 观察要点：")
print("  - 模型是否正确识别了混合情感？")
print("  - 是否提取了所有关键词（价格、客服）？")
print("  - XML 格式是否严格遵循示例？")

实验三：Few-shot 复杂情感分析
测试评论：
价格太贵了，完全不值这个价！不过客服态度还不错，问题解决得挺快。

🤖 模型回复: <review>
  <sentiment>中立</sentiment>
  <keywords>
    <keyword>价格</keyword>
    <keyword>客服态度</keyword>
  </keywords>
</review>

✅ Few-shot 情感分析完成

💡 观察要点：
  - 模型是否正确识别了混合情感？
  - 是否提取了所有关键词（价格、客服）？
  - XML 格式是否严格遵循示例？


---

# 🎓 总结与最佳实践

## 📊 三种格式对比总结

| 格式 | 优势 | 劣势 | 最佳实践 | 典型用例 |
|-----|------|------|---------|---------|
| **XML** | • 语义清晰<br>• 层次分明<br>• 适合嵌套 | • 冗长<br>• 解析稍慢 | • 使用语义化标签<br>• 提供完整示例 | • 文档分析<br>• 复杂数据结构<br>• Claude API |
| **JSON** | • 轻量简洁<br>• 解析快速<br>• API 友好 | • 格式严格<br>• 模型易加 ``` | • 明确禁止 Markdown<br>• 示例要规范 | • 数据提取<br>• API 对接<br>• 配置文件 |
| **HTML** | • 直接展示<br>• 富文本支持 | • 标签复杂<br>• 不利存储 | • 适度使用标签<br>• 注重语义 | • 邮件模板<br>• 网页生成<br>• 内容发布 |

## 💡 格式控制的黄金法则

### 1. **示例 > 描述**
```python
❌ 错误：\"请以 JSON 格式输出，包含 name、age、skills 字段\"
✅ 正确：提供一个完整的 JSON 示例，让模型模仿
```

### 2. **明确禁止额外输出**
```
重要规则：
1. 只返回 [格式] 内容
2. 不要使用 Markdown 代码块（``` 标记）
3. 不要添加解释文字
```

### 3. **One-shot vs Few-shot 选择**

| 场景 | 推荐策略 |
|-----|---------|
| 简单格式转换 | One-shot（1个示例足够） |
| 复杂结构提取 | Few-shot（2-3个示例覆盖边缘情况） |
| 格式固定的 API | One-shot + 严格规则 |

### 4. **验证与容错**

```python
# 推荐：在代码中处理可能的格式问题
try:
    # 清理可能的 Markdown 标记
    clean_output = result.replace('```json', '').replace('```', '').strip()
    parsed_data = json.loads(clean_output)
except:
    # 重试或使用更严格的提示词
    pass
```

## ⚠️ 常见问题与解决方案

| 问题 | 原因 | 解决方案 |
|-----|------|---------|
| JSON 被包裹在 \`\`\`json ... \`\`\` 中 | 模型的 Markdown 倾向 | 明确禁止 Markdown 标记 |
| XML 标签不一致 | 示例不够清晰 | 提供标准示例，强调标签名 |
| 输出包含解释文字 | 提示词不够严格 | 添加\"只输出格式，不要解释\" |
| 格式偶尔不稳定 | One-shot 不够 | 增加到 Few-shot，覆盖边缘情况 |

## 🚀 进阶技巧

### 1. **结合 Pydantic 或 JSON Schema**
```python
# 在提示词中提供 JSON Schema，更严格地约束输出
schema = {
  \"type\": \"object\",
  \"properties\": {
    \"name\": {\"type\": \"string\"},
    \"age\": {\"type\": \"integer\"}
  },
  \"required\": [\"name\", \"age\"]
}
```

### 2. **XML + JSON 混合**
```xml
<data>
  <metadata>...</metadata>
  <content type=\"json\">
    {\"key\": \"value\"}
  </content>
</data>
```
XML 外层提供结构，JSON 内嵌提供数据。

### 3. **动态格式选择**
```python
# 根据任务复杂度动态选择格式
if task_complexity == 'simple':
    format_type = 'JSON'
elif task_complexity == 'complex':
    format_type = 'XML'
```

## 📈 实际应用场景

### RAG 系统中的格式控制
```xml
<rag_response>
  <sources>
    <source id=\"1\">文档A</source>
    <source id=\"2\">文档B</source>
  </sources>
  <answer>基于文档A和B的综合回答...</answer>
  <confidence>0.87</confidence>
</rag_response>
```

### AI Agent 工具调用
```json
{
  \"tool\": \"send_email\",
  \"parameters\": {
    \"to\": \"user@example.com\",
    \"subject\": \"会议提醒\",
    \"body\": \"明天下午2点会议\"
  }
}
```

---

## 🎉 实验完成

通过三个实验，你已经掌握了：

✅ 如何用 XML 输出复杂结构化数据  
✅ 如何用 JSON 进行 API 数据交换  
✅ 如何用 Few-shot 提升格式稳定性  

**关键要点**：
- **示例是最好的指令** - 一个标准示例胜过千言万语
- **明确禁止额外输出** - 让模型只返回格式化数据
- **验证与容错** - 在代码中处理可能的格式问题

💡 **下一步建议**：
- 在实际项目中应用这些格式控制技巧
- 建立自己的\"格式模板库\"
- 结合 RAG 和 Agent 系统实践